## Systemy zlozone - Politechnika Krakowska 2026, SIiRRz

### Autorzy: Mateusz Wójtowicz SIiRRz2, Jakub Kieliński SIiRRz1

Epidemiologiczne  modele  agentowe.  Symulacja  rozprzestrzeniania  się  epidemii  w 
zależności od cech agentów, struktury społecznej i zaraźliwości choroby. 

Importowanie niezbednych bibliotek

In [1]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional

import numpy as np
import matplotlib.pyplot as plt

import networkx as nx
from matplotlib import animation

Parametry glowne modelu

In [2]:
# Reproducibility
RANDOM_SEED = 42  # Stałe ziarno losowości: te same parametry -> podobne wyniki

# Population / simulation horizon
N_AGENTS = 800            # Liczba agentów (węzłów) w sieci
T_MAX = 250               # Maksymalna liczba kroków czasowych (np. "dni")
N_INIT_INFECTED = 3       # Liczba początkowo zakażonych agentów

# Epidemic dynamics (SIR)
BETA_BASE = 0.045         # Bazowe prawdopodobieństwo transmisji na kontakt w 1 kroku
GAMMA = 0.06              # Prawdopodobieństwo wyzdrowienia w 1 kroku (I -> R)

# Optional: heterogeneity in agents (multipliers)
USE_HETEROGENEITY = True  # Czy używać zróżnicowania agentów (podatność/ostrożność)?
SUSC_MEAN = 1.0           # Średni mnożnik podatności (S) na infekcję
SUSC_STD = 0.20           # Odchylenie std podatności (im większe, tym większa różnorodność)

BEHAVIOR_MEAN = 1.0       # Średni mnożnik zachowania (1.0 = brak redukcji kontaktu)
BEHAVIOR_STD = 0.25       # Odchylenie std zachowania (np. ostrożni <1.0)

# Network selection
NETWORK_TYPE = "WS"       # "ER" | "BA" | "WS" | "SBM"

# Network parameters
# ER: p = probability of edge
ER_P = 0.015

# BA: m = number of edges each new node attaches
BA_M = 6

# WS: k = each node is connected to k nearest neighbors in ring; p_rewire = rewiring probability
WS_K = 10
WS_P_REWIRE = 0.08

# SBM: community sizes + within/between probabilities
SBM_SIZES = [200, 200, 200, 200]  # Suma musi dawać N_AGENTS (tu: 800)
SBM_P_IN = 0.030                  # Prawdopodobieństwo połączeń wewnątrz community
SBM_P_OUT = 0.0025                # Prawdopodobieństwo połączeń między community

# Monte Carlo / repeats (for later experiments)
N_RUNS = 10  # Ile powtórzeń symulacji dla uśredniania wyników

# Plotting
EPS_LOG = 1e-6  # Mała stała, aby wykres logarytmiczny działał nawet dla 0
FIG_DPI = 120

# GIF settings
MAKE_GIF = True
GIF_EVERY = 2           # Co ile kroków zapisywać klatkę do GIF (większe = mniejszy GIF)
GIF_FPS = 12
GIF_LAYOUT = "spring"   # "spring" lub "kamada_kawai" (układ grafu do wizualizacji)

np.random.seed(RANDOM_SEED)

Funkcje pomocnicze: skala log + formatowanie

In [3]:
def safe_log_values(x: np.ndarray, eps: float = EPS_LOG) -> np.ndarray:
    """Return x + eps for log-scale plotting (avoids log(0))."""
    return np.asarray(x) + eps


def english_title(s: str) -> str:
    """Small helper to enforce English titles consistently."""
    return str(s)

Generowanie sieci kontaktów (ER/BA/WS/SBM)

In [4]:
def build_network(
    network_type: str,
    n_agents: int,
    seed: int = RANDOM_SEED
) -> nx.Graph:
    network_type = network_type.upper()

    if network_type == "ER":
        G = nx.erdos_renyi_graph(n_agents, ER_P, seed=seed)

    elif network_type == "BA":
        # BA wymaga m < n
        m = min(BA_M, max(1, n_agents - 1))
        G = nx.barabasi_albert_graph(n_agents, m, seed=seed)

    elif network_type == "WS":
        # WS wymaga parzystego k i k < n
        k = min(WS_K, n_agents - 1)
        if k % 2 == 1:
            k += 1
        if k >= n_agents:
            k = n_agents - 1 if (n_agents - 1) % 2 == 0 else n_agents - 2
        G = nx.watts_strogatz_graph(n_agents, k, WS_P_REWIRE, seed=seed)

    elif network_type == "SBM":
        if sum(SBM_SIZES) != n_agents:
            raise ValueError(f"SBM_SIZES sum must be {n_agents}, got {sum(SBM_SIZES)}")

        # macierz prawdopodobieństw bloków
        k = len(SBM_SIZES)
        p = np.full((k, k), SBM_P_OUT, dtype=float)
        np.fill_diagonal(p, SBM_P_IN)

        G = nx.stochastic_block_model(SBM_SIZES, p, seed=seed)

        # Ujednolicamy etykiety węzłów do 0..N-1 (czasem SBM daje już takie, ale wolimy pewność)
        G = nx.convert_node_labels_to_integers(G)

    else:
        raise ValueError("NETWORK_TYPE must be one of: ER, BA, WS, SBM")

    # Jeśli graf jest rozłączny, to w epidemii mogą powstać "wyspy".
    # Zostawiamy to (to też ciekawy efekt), ale możemy ostrzec.
    return G


G = build_network(NETWORK_TYPE, N_AGENTS, seed=RANDOM_SEED)
print(f"Network: {NETWORK_TYPE}, nodes={G.number_of_nodes()}, edges={G.number_of_edges()}, connected={nx.is_connected(G)}")


Network: WS, nodes=800, edges=4000, connected=True
